In [33]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class MIDIVAE(nn.Module):
  def __init__(self, encoder,decoder,classifier,device = 'cuda'):
      super(MIDIVAE,self).__init__()


      # untrained components
      self.encoder = encoder
      self.decoder = decoder

      # pre-trained classifier
      self.classifer = classifier


      self.device = device

  def forward(self, x,label):
    #obtain latent space
    z, mean,logvariance = self.encoder(x)
    # concatenate x label with z
    z = torch.cat([z,label.to(self.device)])
    #feed through decoder
    recon_midi = self.decoder(z)

    #feed through classifier
    composer_pred = self.classifier(recon_midi)

    return recon_midi,mean,logvariance,composer_pred

  def train(self, dataloader, optimizer, epochs=10, device='cuda'):
    self.train()
    # load data into some kind of trainloader?

    # for loop with loading data for each batch
    for epoch in range(epochs):
        total_loss = 0
        for x, label, composer in dataloader:  # assume dataloader returns inputs, labels, composers
            x, label, composer = x.to(device), label.to(device), composer.to(device)

            optimizer.zero_grad()

            # call forward and get back the reconstructions, mean, logvariance, and composer prediction

            recon_x, mu, logvar, pred = self.forward(x, label)

            # do KLD with mean, logvariance and our prior N(0,I), backprop to encoder only
            kld = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) / x.size(0)

            # do MSE with reconstructions and original data, backprop to encoder & decoder

            recon_loss = F.mse_loss(recon_x, x)

            # do CrossEntropy with data labels and composer predictions, backprop to decoder only

            ce_loss = F.cross_entropy(pred, composer)

            loss = recon_loss + kld + ce_loss  # optionally weigh each

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f"Epoch {epoch+1} | Loss: {total_loss:.4f}")

  def generate(self,label):
    # put on test mode
    self.eval()
    # generate normal gaussian noise
    batch_size = label.size(0)
    z = torch.randn(batch_size, self.encoder.latent_proj.fc_mu.out_features).to(self.device)
    # concatenate noise with label
    z_cond = torch.cat([z, label.to(self.device)], dim=1)
    # feed through decoder

    recon = self.decoder(z_cond)
    return recon


Token Embedding + Positional Encoding
           ↓
  Transformer Encoder
           ↓
     Sequence Embeddings (B, T, D)
           ↓
    Mean Pool / CLS Token → (B, D)
           ↓
      Linear → mu (B, latent_dim)
      Linear → logvar (B, latent_dim)
           ↓
Reparameterization Trick: z = mu + eps * std

label → Embedding → (B, label_dim)
z     → (B, latent_dim)

→ concat [z, label_emb] → (B, latent_dim + label_dim)
→ Linear projection → (B, decoder_dim)  # matches decoder embedding dim
→ z_cond

[BOS, ..., tokens[:t-1]] → token embedding + pos encoding → (B, T', D)

z_cond → unsqueeze(1) → broadcast across T' → (B, T', D)

decoder_input = token_emb + z_cond

Transformer Decoder → token_logits (B, T', vocab_size)


In [28]:

class Token_Embedding(nn.Module):
  def __init__(self,vocab_size,embedding_dim):
    super(Token_Embedding,self).__init__()
    self.embedding = nn.Embedding(vocab_size,embedding_dim)

  def forward(self,x):
    return self.embedding(x)

class Pos_Embedding(nn.Module):
  def __init__(self,max_len,embedding_dim):
    super(Pos_Embedding,self).__init__()
    self.pos_embedding = nn.Embedding(max_len,embedding_dim)

  def forward(self,x):
    seq_len = x.size(1)
    pos_ids = torch.arange(seq_len,device=x.device).unsqueeze(0)
    return self.pos_embedding(pos_ids)

class Transformer_Encoder(nn.Module):
  def __init__(self,embedding_dim,num_heads,num_layers,ff_dim):
    super(Transformer_Encoder,self).__init__()
    self.self_attention = nn.MultiheadAttention(embedding_dim,num_heads)
    self.layer_norm = nn.LayerNorm(embedding_dim)
    self.ffn = nn.Sequential(nn.Linear(embedding_dim,ff_dim),nn.ReLU(),nn.Linear(ff_dim,embedding_dim))

  def forward(self,x):
    print(x.shape)
    # multihead attention is dumb and doesnt like batch size first
    x = x.permute(1, 0, 2)
    output,_ = self.self_attention(x,x,x)
    # maybe add normalization

    ffn_output = self.ffn(x)
    x = self.layer_norm(x + ffn_output)
    #undo what we did
    x = x.permute(1, 0, 2)
    return x

class LatentSpace_Mean_Log(nn.Module):
  def __init__(self,embedding_dim,latent_dim):
    super(LatentSpace_Mean_Log,self).__init__()
    self.fc_mu = nn.Linear(embedding_dim,latent_dim)
    self.fc_logvar = nn.Linear(embedding_dim,latent_dim)

  def forward(self,x):
    mu = self.fc_mu(x)
    logvar = self.fc_logvar(x)

    return mu,logvar

In [30]:
class Variational_Encoder(nn.Module):
  def __init__(self, vocab_size, embedding_dim, max_len, latent_dim, num_heads, num_layers, ff_dim):
      super(Variational_Encoder,self).__init__()

      self.token_embedding = Token_Embedding(vocab_size,embedding_dim)
      self.pos_embedding = Pos_Embedding(max_len,embedding_dim)
      self.encoder = Transformer_Encoder(embedding_dim,num_heads,num_layers,ff_dim)
      self.latent_proj = LatentSpace_Mean_Log(embedding_dim,latent_dim)

  def forward(self, x):
    # create embeddings
    tok_embeddings = self.token_embedding(x)
    pos_embeddings = self.pos_embedding(x)
    embeddings = pos_embeddings + tok_embeddings

    # obtain output
    output = self.encoder(embeddings)

    # pool it up!!
    pooled_output = output.mean(dim=1)

    # project into latent space and reparameterize
    mean,logvariance = self.latent_proj(pooled_output)
    std = torch.exp(0.5 * logvariance)
    eps = torch.randn_like(std)
    z = mean + eps * std

    return z, mean,logvariance


In [31]:
# example of instantiation of it

vocab_size = 512        # Number of unique MIDI tokens
embedding_dim = 256     # Size of token embeddings
max_len = 512          # Max sequence length (you can adjust this)
latent_dim = 128       # Latent space dimension
num_heads = 8          # Number of attention heads
num_layers = 6         # Number of transformer layers
ff_dim = 512           # Feed-forward layer dimension

encoder = Variational_Encoder(vocab_size, embedding_dim, max_len, latent_dim, num_heads, num_layers, ff_dim)

# Example input (batch of MIDI token sequences)
x = torch.randint(0, vocab_size, (32, 100))  # Batch size 32, sequence length 100

z, mu, logvar = encoder(x)
print(f"z shape: {z.shape}, mu shape: {mu.shape}, logvar shape: {logvar.shape}")


torch.Size([32, 100, 256])
z shape: torch.Size([32, 128]), mu shape: torch.Size([32, 128]), logvar shape: torch.Size([32, 128])
